In [1]:
# This code is for simulation polarimetric calibration, after correction for in-homogeneity.

import cv2
import polanalyser as pa
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from mpldatacursor import datacursor

In [12]:
#Simulate groud truth normalized Stokes Vector
DoLP_true = 0.999
haxis = 10
vaxis = 10
n_im = 37
I_source = np.ones([n_im,vaxis,haxis])
t_guess = 1
q_true_val = np.zeros(n_im)
u_true_val = np.zeros(n_im)
q_true = np.ones([n_im,vaxis,haxis])
u_true = np.ones([n_im,vaxis,haxis])
AoLP_true = np.deg2rad(range(0,185,5))
q_true_val = DoLP_true*np.cos(2*AoLP_true)
u_true_val = DoLP_true*np.sin(2*AoLP_true)
for i in range(n_im):
    q_true[i,:,:] = q_true_val[i]
    u_true[i,:,:] = u_true_val[i]

In [13]:
#Simulate mono images, no demosaicing
alpha_d = np.ones([n_im,vaxis,haxis])
alpha_d[:,1::2, 1::2]=  0
alpha_d[:,0::2, 1::2]=  45
alpha_d[:,0::2, 0::2]=  90
alpha_d[:,1::2, 0::2]=  135
alpha = np.deg2rad(alpha_d)
L = 0.5*t_guess*(I_source+ q_true*np.cos(2*alpha)+u_true*np.sin(2*alpha))
noise_std = 5
noise =  np.random.normal(0.0,noise_std/100,L.shape)
L_noise = L*(1+noise)

In [ ]:
#Show noisy images
for i in range(n_im):
    plt.figure(1)
    im1 = plt.imshow(L_noise[i,:,:])
    im = Image.fromarray(L_noise[i,:,:])
    im.save(f'{i}.tiff')
    plt.colorbar(im1)
    datacursor()
    plt.title('I')
    plt.axis('off')
    plt.show()

In [14]:
def mean_error(images, calibrate = False):
    n_im = images.shape[0]
    DoLP_Error = np.zeros(n_im)
    AoLP_Error = np.zeros(n_im)
    AoLP_sum = 0
    for i in range(n_im):
        images_demosaiced = pa.demosaicing(images[i])
        img_0, img_45, img_90, img_135 = cv2.split(images_demosaiced)
        Stokes = pa.calcLinearStokes(np.array([img_0, img_45, img_90, img_135]), np.deg2rad([0,45,90,135]))
        if calibrate:
            X_mat = Cal_params(images)
            Stokes_cal = Cal(Stokes, X_mat)
            Stokes = Stokes_cal
        DoLP_with_noise= pa.cvtStokesToDoLP(Stokes)
        AoLP_with_noise = pa.cvtStokesToAoLP(Stokes)
        DoLP_Error[i] = np.sum(np.abs(DoLP_with_noise-DoLP_true))
        AoLP_Error[i] = np.sum(np.abs(AoLP_with_noise-AoLP_true[i]))
        AoLP_sum +=  haxis*vaxis*AoLP_true[i]
    mean_DoLP_Error = np.sum(DoLP_Error)/(n_im*haxis*vaxis*DoLP_true)
    mean_AoLP_Error = np.sum(AoLP_Error)/AoLP_sum
    return [mean_DoLP_Error, mean_AoLP_Error]
m_error_0 = mean_error(L_noise,n_im)

In [20]:
def Cal_params(images):
    #calibration

    n_im = images.shape[0]
    haxis = images.shape[1]
    vaxis = images.shape[2]

    images = np.zeros([n_im,haxis,vaxis])
    B_Mat = np.zeros([n_im,haxis*vaxis])
    A_Mat = np.zeros([n_im, 3])

    for i in range(n_im):
        B_Mat[i] = images[i].flatten()

    A_Mat[:,0]=1
    A_Mat[:,1] = np.transpose(q_true_val)
    A_Mat[:,2] = np.transpose(u_true_val)

    # for real images we need normalization
    #    I = 0.5*(images[:,1::2, 1::2]+images[:,0::2, 1::2]+images[:,0::2, 0::2]+images[:,1::2, 0::2])
    #    images[:,1::2, 1::2]=  images[:,1::2, 1::2]/I
    #    images[:,0::2, 1::2]=  images[:,0::2, 1::2]/I
    #    images[:,0::2, 0::2]=  images[:,0::2, 0::2]/I
    #    images[:,1::2, 0::2]=  images[:,1::2, 0::2]/I

    x = np.linalg.lstsq(A_Mat, B_Mat)
    X_mat = np.reshape(np.array(x[0]),[3,haxis,vaxis])# each column has the three parameters for a pixel
    return X_mat

<ipython-input-20-d3d32b3c15c2>:26: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  x = np.linalg.lstsq(A_Mat, B_Mat)


In [ ]:
def Cal(Stokes, X_mat):
    images_demosaiced = pa.demosaicing(X[i])
    img_0, img_45, img_90, img_135 = cv2.split(images_demosaiced)

In [ ]:
for i in range(n_im):
    images[i] = np.asarray(Image.open(f'{i}.tiff'))
